In [ ]:
import pygame
import chess
import sys

# === CONFIG ===
WIDTH, HEIGHT = 512, 512
SQ_SIZE = WIDTH // 8
FPS = 60
IMAGES = {}

# === LOAD IMAGES ===
def load_images():
    pieces = ['P', 'R', 'N', 'B', 'Q', 'K']
    for color in ['w', 'b']:
        for piece in pieces:
            name = color + piece
            IMAGES[name] = pygame.transform.scale(
                pygame.image.load(f'images/{name}.png'), (SQ_SIZE, SQ_SIZE)
            )

# === DRAWING FUNCTIONS ===
def draw_board(screen):
    colors = [pygame.Color("white"), pygame.Color("gray")]
    for r in range(8):
        for c in range(8):
            color = colors[(r + c) % 2]
            pygame.draw.rect(screen, color, pygame.Rect(c*SQ_SIZE, r*SQ_SIZE, SQ_SIZE, SQ_SIZE))

def draw_pieces(screen, board, dragging_piece, drag_pos):
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece and square != dragging_piece:
            col = chess.square_file(square)
            row = 7 - chess.square_rank(square)
            key = f"{'w' if piece.color == chess.WHITE else 'b'}{piece.symbol().upper()}"
            screen.blit(IMAGES[key], pygame.Rect(col * SQ_SIZE, row * SQ_SIZE, SQ_SIZE, SQ_SIZE))

    if dragging_piece is not None:
        piece = board.piece_at(dragging_piece)
        if piece:
            key = f"{'w' if piece.color == chess.WHITE else 'b'}{piece.symbol().upper()}"
            piece_img = IMAGES[key]
            screen.blit(piece_img, piece_img.get_rect(center=drag_pos))

def highlight_squares(screen, board, selected_square):
    if selected_square is None:
        return

    legal_moves = [move for move in board.legal_moves if move.from_square == selected_square]
    s_col = chess.square_file(selected_square)
    s_row = 7 - chess.square_rank(selected_square)
    pygame.draw.rect(screen, pygame.Color("green"), pygame.Rect(s_col*SQ_SIZE, s_row*SQ_SIZE, SQ_SIZE, SQ_SIZE), 4)

    for move in legal_moves:
        t_col = chess.square_file(move.to_square)
        t_row = 7 - chess.square_rank(move.to_square)
        center = (t_col * SQ_SIZE + SQ_SIZE//2, t_row * SQ_SIZE + SQ_SIZE//2)
        pygame.draw.circle(screen, pygame.Color("green"), center, 10)

# === AI FUNCTIONS ===
def evaluate_board(board):
    if board.is_checkmate():
        return -9999 if board.turn else 9999
    if board.is_stalemate():
        return 0
    values = {chess.PAWN: 100, chess.KNIGHT: 320, chess.BISHOP: 330,
              chess.ROOK: 500, chess.QUEEN: 900, chess.KING: 0}
    score = 0
    for piece_type in values:
        score += len(board.pieces(piece_type, chess.WHITE)) * values[piece_type]
        score -= len(board.pieces(piece_type, chess.BLACK)) * values[piece_type]
    return score

def alphabeta(board, depth, alpha, beta, maximizing):
    if depth == 0 or board.is_game_over():
        return evaluate_board(board)
    if maximizing:
        max_eval = -float('inf')
        for move in board.legal_moves:
            board.push(move)
            eval = alphabeta(board, depth - 1, alpha, beta, False)
            board.pop()
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for move in board.legal_moves:
            board.push(move)
            eval = alphabeta(board, depth - 1, alpha, beta, True)
            board.pop()
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

def find_best_move(board, depth):
    best_move = None
    best_value = -float('inf') if board.turn else float('inf')
    for move in board.legal_moves:
        board.push(move)
        value = alphabeta(board, depth - 1, -float('inf'), float('inf'), not board.turn)
        board.pop()
        if board.turn and value > best_value:
            best_value = value
            best_move = move
        elif not board.turn and value < best_value:
            best_value = value
            best_move = move
    return best_move

# === MAIN LOOP ===
def main():
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("ChessBot GUI (Drag-and-Drop)")
    clock = pygame.time.Clock()
    load_images()

    board = chess.Board()
    selected_square = None
    dragging = False
    dragging_piece = None
    drag_pos = (0, 0)

    running = True
    while running:
        draw_board(screen)
        highlight_squares(screen, board, selected_square)
        draw_pieces(screen, board, dragging_piece, drag_pos)
        pygame.display.flip()

        if board.turn == chess.BLACK and not board.is_game_over():
            pygame.time.wait(300)
            move = find_best_move(board, 3)
            if move:
                board.push(move)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            elif event.type == pygame.MOUSEBUTTONDOWN and board.turn == chess.WHITE:
                x, y = event.pos
                col = x // SQ_SIZE
                row = 7 - (y // SQ_SIZE)
                square = chess.square(col, row)
                if board.piece_at(square) and board.piece_at(square).color == chess.WHITE:
                    selected_square = square
                    dragging = True
                    dragging_piece = square
                    drag_pos = event.pos

            elif event.type == pygame.MOUSEMOTION and dragging:
                drag_pos = event.pos

            elif event.type == pygame.MOUSEBUTTONUP and dragging:
                x, y = event.pos
                to_col = x // SQ_SIZE
                to_row = 7 - (y // SQ_SIZE)
                to_square = chess.square(to_col, to_row)
                move = chess.Move(selected_square, to_square)
                if move in board.legal_moves:
                    board.push(move)
                selected_square = None
                dragging = False
                dragging_piece = None

        if board.is_game_over():
            print("Game Over:", board.result())
            pygame.time.wait(2000)
            running = False

        clock.tick(FPS)

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()


SystemExit: 